# Analyze on LLM result from micros benchmarks

## Init Datasets

## What do we want to get

1. Overall CWE it found
2. CWE Count
3. CWE real exists
4. Double check by other GPT again
5. CWE exists in each repo
6. Real CWE
7. real CWE / CWE in each repo

In [5]:
:dep plotters = { version = "0.3", default_features = false, features = ["evcxr", "all_series"] }
:dep polars = { version = "0.41.3", features = ["serde","serde-lazy","lazy","simd","avx512","performant","temporal","json","strings","regex","rows","pivot"] }
:dep mimalloc = { version = "0.1.43" }
:dep anyhow = { version = "1.0.86", features = ["backtrace"] }
:dep serde = { version = "1.0.203", features = ["derive"] }
:dep serde_json = { version = "1.0.120" }
:dep rayon = "1.10.0"
:dep regex = "1.10.5"

In [15]:
use pivot::pivot;
use polars::{lazy::dsl::col, prelude::*};
use std::{fs::File, path::Path};

fn init() -> anyhow::Result<DataFrame> {
    let path = "../micros.3.log";
    let file = File::open(path)?;
    let fields = [
        Field::new("cweID", DataType::String),
        Field::new("fileName", DataType::String),
        Field::new("codeWithIssue", DataType::String),
        Field::new("time", DataType::String),
        Field::new("content", DataType::String),
        Field::new("msg", DataType::String),
        Field::new("path", DataType::String),
        Field::new("level", DataType::Int64),
        Field::new("time", DataType::Int64),
        Field::new("hostname", DataType::String),
        Field::new("pir", DataType::Int64),
    ];
    let schema = Schema::from_iter(fields.into_iter());
    let df: DataFrame = JsonLineReader::new(file)
        .with_schema(Arc::new(schema))
        .finish()?;
    let df = df
        .lazy()
        .filter(col("cweID").is_not_null())
        .with_column(
            col("cweID")
                .str()
                .extract(lit(r"(CWE-\d+)"), 1)
                .alias("cwe_id"),
        )
        .with_column(
            col("fileName")
                .str()
                .extract(lit(r"micros/([^/]+)/"), 1)
                .alias("repo"),
        )
        .collect()?
        .select(["cwe_id", "repo", "fileName", "codeWithIssue"])?;
    Ok(df)
}

In [16]:
let df = init().unwrap();
let overall_files = df.height();

In [17]:
let df = df
    .lazy()
    .filter(col("cwe_id").str().contains(lit("CWE-\\d+"), false))
    .collect().unwrap();
let cwes = df.height();
println!("{} / {}", cwes, overall_files);

8035 / 12138


In [ ]:
let df = df
    .lazy()
    .group_by(["repo"])
    .agg([col("cwe_id")
        .value_counts(true, false, "cwe_id_occur".to_owned(), false)
        .alias("cwe_id_occur")])
    .explode(["cwe_id_occur"])
    .unnest(["cwe_id_occur"])
    .collect().unwrap();
let pivot_df = pivot(
    &df,
    ["repo"],
    Some(["cwe_id"]),
    Some(["cwe_id_occur"]),
    true,
    None,
    None,
)
.unwrap()
.fill_null(FillNullStrategy::Zero)
.unwrap();
